In [8]:
from PIL import Image
import os

In [9]:
input_dir = "./data/raw/full/output_normalized"
output_dir = "./data/prepared"
os.makedirs(output_dir, exist_ok=True)

def pad_image(img, target_w, target_h):
    new_img = Image.new("RGB", (target_w, target_h), (255, 255, 255))
    new_img.paste(img, ((target_w - img.width) // 2, (target_h - img.height) // 2))
    return new_img

# --- อ่านไฟล์และ sort ---
files = sorted([f for f in os.listdir(input_dir) if f.endswith("_mangosteen_1.jpg")])

# Group images by index (i001, i002, ...)
groups = {}
for f in files:
    idx = f.split("_")[0]  # เช่น i001
    if idx not in groups:
        groups[idx] = {}
    if "upper" in f:
        groups[idx]["upper"] = f
    elif "under" in f:
        groups[idx]["under"] = f
    elif "side" in f:
        groups[idx]["side"] = f

# --- รวมภาพแต่ละกลุ่ม ---
for idx, imgs in sorted(groups.items()):
    if not all(k in imgs for k in ["upper", "under", "side"]):
        print(f"⚠️ Skip {idx}, missing images")
        continue

    upper = Image.open(os.path.join(input_dir, imgs["upper"]))
    under = Image.open(os.path.join(input_dir, imgs["under"]))
    side = Image.open(os.path.join(input_dir, imgs["side"]))
    side_flip = side.transpose(Image.Transpose.FLIP_LEFT_RIGHT)

    # Normalize ขนาด
    max_w = max(img.width for img in [upper, under, side, side_flip])
    max_h = max(img.height for img in [upper, under, side, side_flip])
    upper = pad_image(upper, max_w, max_h)
    under = pad_image(under, max_w, max_h)
    side = pad_image(side, max_w, max_h)
    side_flip = pad_image(side_flip, max_w, max_h)

    # รวมเป็น grid
    combined = Image.new("RGB", (max_w*2, max_h*2), (255, 255, 255))
    combined.paste(upper, (0, 0))
    combined.paste(under, (max_w, 0))
    combined.paste(side, (0, max_h))
    combined.paste(side_flip, (max_w, max_h))

    # Resize 224x224
    final_img = combined.resize((224, 224))
    output_path = os.path.join(output_dir, f"{idx}_mangosteen_grid.jpg")
    final_img.save(output_path)

    print(f"✅ Saved {output_path}")


✅ Saved ./data/prepared/i001_mangosteen_grid.jpg
✅ Saved ./data/prepared/i002_mangosteen_grid.jpg
✅ Saved ./data/prepared/i003_mangosteen_grid.jpg
✅ Saved ./data/prepared/i004_mangosteen_grid.jpg
✅ Saved ./data/prepared/i005_mangosteen_grid.jpg
✅ Saved ./data/prepared/i006_mangosteen_grid.jpg
✅ Saved ./data/prepared/i007_mangosteen_grid.jpg
✅ Saved ./data/prepared/i008_mangosteen_grid.jpg
✅ Saved ./data/prepared/i009_mangosteen_grid.jpg
✅ Saved ./data/prepared/i010_mangosteen_grid.jpg
✅ Saved ./data/prepared/i011_mangosteen_grid.jpg
✅ Saved ./data/prepared/i012_mangosteen_grid.jpg
✅ Saved ./data/prepared/i013_mangosteen_grid.jpg
✅ Saved ./data/prepared/i014_mangosteen_grid.jpg
✅ Saved ./data/prepared/i015_mangosteen_grid.jpg
✅ Saved ./data/prepared/i016_mangosteen_grid.jpg
✅ Saved ./data/prepared/i017_mangosteen_grid.jpg
✅ Saved ./data/prepared/i018_mangosteen_grid.jpg
✅ Saved ./data/prepared/i019_mangosteen_grid.jpg
✅ Saved ./data/prepared/i020_mangosteen_grid.jpg
✅ Saved ./data/prepa